In [1]:
import pandas as pd

# Load the dataset
df = pd.read_excel("..//..//datasets//modified_location_9.xlsx")  # Replace with your actual file path

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')  # Adjust format if necessary

# Drop rows with missing values in 'Rainfall' or 'Date'
df.dropna(subset=['Rainfall (in mm)'], inplace=True)

# Select only the 'Date' and 'Rainfall' columns
df = df[['Date', 'Rainfall (in mm)']]

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Convert Rainfall to time series (assuming daily data)
rainfall_ts = df['Rainfall (in mm)']

# Display the first few rows to check the data
print(df.head())

            Rainfall (in mm)
Date                        
1901-01-01               3.0
1901-01-02               8.0
1901-01-03               0.0
1901-01-04               0.0
1901-01-05               0.0


In [2]:
print(df.shape)  # (rows, columns)

(61600, 1)


In [3]:
from pmdarima import auto_arima

# Define training range
train_start = '2018-01-01'
train_end = '2022-12-31'

# Slice training data
y_train = rainfall_ts[train_start:train_end]

# Fit a lightweight seasonal ARIMA model
model = auto_arima(
    y_train,
    seasonal=True,
    m=15,  
    start_p=0, start_q=0, max_p=1, max_q=1, max_d=1,
    max_P=1, max_Q=1, max_D=1,
    stepwise=True,
    trace=True,
    suppress_warnings=True,
    information_criterion='aic'
)

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[15] intercept   : AIC=29217.034, Time=3.39 sec
 ARIMA(0,0,0)(0,0,0)[15] intercept   : AIC=29710.473, Time=0.07 sec
 ARIMA(1,0,0)(1,0,0)[15] intercept   : AIC=26471.983, Time=1.90 sec
 ARIMA(0,0,1)(0,0,1)[15] intercept   : AIC=26637.029, Time=3.80 sec
 ARIMA(0,0,0)(0,0,0)[15]             : AIC=31061.773, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[15] intercept   : AIC=26474.201, Time=0.21 sec
 ARIMA(1,0,0)(1,0,1)[15] intercept   : AIC=26459.830, Time=3.79 sec
 ARIMA(1,0,0)(0,0,1)[15] intercept   : AIC=26472.305, Time=1.46 sec
 ARIMA(1,0,1)(1,0,1)[15] intercept   : AIC=26262.686, Time=9.30 sec
 ARIMA(1,0,1)(0,0,1)[15] intercept   : AIC=26302.844, Time=5.33 sec
 ARIMA(1,0,1)(1,0,0)[15] intercept   : AIC=26301.380, Time=2.46 sec
 ARIMA(1,0,1)(0,0,0)[15] intercept   : AIC=26311.327, Time=0.82 sec
 ARIMA(0,0,1)(1,0,1)[15] intercept   : AIC=26484.544, Time=8.38 sec
 ARIMA(1,0,1)(1,0,1)[15]             : AIC=26372.560, Time=2.89 sec

Best

In [4]:
# Forecast for 2023
forecast_start = '2023-01-01'
forecast_end = '2023-12-31'

# Extract actual values
y_true_2023 = df[(df.index >= forecast_start) & (df.index <= forecast_end)]['Rainfall (in mm)']

# Predict for the same period
n_periods = len(y_true_2023)
y_pred_2023 = model.predict(n_periods=n_periods)

# Check lengths
print("Actual data size:", len(y_true_2023))
print("Predicted data size:", len(y_pred_2023))

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Evaluation metrics
mse_2023 = mean_squared_error(y_true_2023, y_pred_2023)  # MSE instead of RMSE
mae_2023 = mean_absolute_error(y_true_2023, y_pred_2023)
r2_2023 = r2_score(y_true_2023, y_pred_2023)

print(f"MSE for 2023 forecast: {mse_2023}")
print(f"MAE for 2023 forecast: {mae_2023}")
print(f"R² for 2023 forecast: {r2_2023}")


# Create results DataFrame using index (dates) directly
forecast_df_2023 = pd.DataFrame({
    'Date': y_true_2023.index,                   # correct date index
    'Actual Rainfall': y_true_2023.values,       # actual values
    'Predicted Rainfall': y_pred_2023            # predicted values
})

# Save to Excel
forecast_df_2023.to_excel('.//sarima_location_9_(15_5yrs).xlsx', index=False)

print("✅ Forecast saved successfully: 2023_forecast_results_Sarima.xlsx")


Actual data size: 365
Predicted data size: 365
MSE for 2023 forecast: 156.1907666394316
MAE for 2023 forecast: 9.114907106481208
R² for 2023 forecast: 0.01747847408766179
✅ Forecast saved successfully: 2023_forecast_results_Sarima.xlsx


C:\Users\rahla\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\rahla\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [5]:
# now with m = 30

In [6]:
from pmdarima import auto_arima

# Define training range
train_start = '2018-01-01'
train_end = '2022-12-31'

# Slice training data
y_train = rainfall_ts[train_start:train_end]

# Fit a lightweight seasonal ARIMA model
model_30 = auto_arima(
    y_train,
    seasonal=True,
    m=30,  
    start_p=0, start_q=0, max_p=1, max_q=1, max_d=1,
    max_P=1, max_Q=1, max_D=1,
    stepwise=True,
    trace=True,
    suppress_warnings=True,
    information_criterion='aic'
)

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[30] intercept   : AIC=29365.452, Time=10.40 sec
 ARIMA(0,0,0)(0,0,0)[30] intercept   : AIC=29710.473, Time=0.07 sec
 ARIMA(1,0,0)(1,0,0)[30] intercept   : AIC=26470.131, Time=7.65 sec
 ARIMA(0,0,1)(0,0,1)[30] intercept   : AIC=26634.782, Time=7.87 sec
 ARIMA(0,0,0)(0,0,0)[30]             : AIC=31061.773, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[30] intercept   : AIC=26474.201, Time=0.21 sec
 ARIMA(1,0,0)(1,0,1)[30] intercept   : AIC=26466.972, Time=11.37 sec
 ARIMA(1,0,0)(0,0,1)[30] intercept   : AIC=26470.624, Time=6.99 sec
 ARIMA(1,0,1)(1,0,1)[30] intercept   : AIC=26287.616, Time=12.93 sec
 ARIMA(1,0,1)(0,0,1)[30] intercept   : AIC=26297.866, Time=9.88 sec
 ARIMA(1,0,1)(1,0,0)[30] intercept   : AIC=26296.136, Time=9.07 sec
 ARIMA(1,0,1)(0,0,0)[30] intercept   : AIC=26311.327, Time=0.97 sec
 ARIMA(0,0,1)(1,0,1)[30] intercept   : AIC=26587.325, Time=16.24 sec
 ARIMA(1,0,1)(1,0,1)[30]             : AIC=inf, Time=22.98 sec

Best 

In [7]:
# Forecast for 2023
forecast_start = '2023-01-01'
forecast_end = '2023-12-31'

# Extract actual values
y_true_2023 = df[(df.index >= forecast_start) & (df.index <= forecast_end)]['Rainfall (in mm)']

# Predict for the same period
n_periods = len(y_true_2023)
y_pred_2023 = model_30.predict(n_periods=n_periods)

# Check lengths
print("Actual data size:", len(y_true_2023))
print("Predicted data size:", len(y_pred_2023))

# Evaluation metrics
mse_2023 = mean_squared_error(y_true_2023, y_pred_2023)
mae_2023 = mean_absolute_error(y_true_2023, y_pred_2023)
r2_2023 = r2_score(y_true_2023, y_pred_2023)

print(f"MSE for 2023 forecast: {mse_2023}")
print(f"MAE for 2023 forecast: {mae_2023}")
print(f"R² for 2023 forecast: {r2_2023}")

# Create results DataFrame using index (dates) directly
forecast_df_2023 = pd.DataFrame({
    'Date': y_true_2023.index,                   # correct date index
    'Actual Rainfall': y_true_2023.values,       # actual values
    'Predicted Rainfall': y_pred_2023            # predicted values
})

# Save to Excel
forecast_df_2023.to_excel('.//sarima_location_9_(30_5yrs.xlsx', index=False)

print("✅ Forecast saved successfully: 2023_forecast_results_Sarima.xlsx")


Actual data size: 365
Predicted data size: 365
MSE for 2023 forecast: 157.3240906710594
MAE for 2023 forecast: 9.264646513870108
R² for 2023 forecast: 0.010349273809910242
✅ Forecast saved successfully: 2023_forecast_results_Sarima.xlsx


C:\Users\rahla\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\rahla\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
